# Predictions of properties from compositions

This notebook allows to load models and predict outputs using it.

In this example, we use the models trained in `Effect_outliers_on_best_models` (in one iteration and one k-fold). 
The compositions of which we want to predict the properties must be written in a csv file `composition2predict`.
Given the path of the model, the notebook proposes to load keras model or scikitlearn models and to give the model predictions. 


## Import libraries

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import json
import joblib

In [11]:
datasets_dir=os.getenv("DATASETS_DIR")
run_dir=os.getenv("RUN_DIR")

## Load compositions of which we want to predict the properties

In [12]:
compo2predict=pd.read_csv(f"{datasets_dir}composition2predict.csv",header=0)*100
display(compo2predict)

,Zr,Nb,Mo,Ti,Cr
0,0.0,0.0,0.0,0.0,100.0
1,0.0,0.0,0.0,2.0,98.0
2,0.0,0.0,0.0,4.0,96.0
3,0.0,0.0,0.0,6.0,94.0
4,0.0,0.0,0.0,8.0,92.0
...,...,...,...,...,...
316246,98.0,0.0,0.0,0.0,2.0
316247,98.0,0.0,0.0,2.0,0.0
316248,98.0,0.0,2.0,0.0,0.0
316249,98.0,2.0,0.0,0.0,0.0


## 2) Predictive models path

In [13]:
phase_EBSD_class_NN=f"{run_dir}campaign09cNN/Class_EBSD/Class_EBSD-0000/iter-000/kfold-00/"
phase_XRD_class_NN=f"{run_dir}campaign09cNN/Class_XRD/Class_XRD-0000/iter-000/kfold-00/"
phase_EBSD_class_RF=f"{run_dir}campaign09c/Class_EBSD/Class_EBSD-0001/iter-000/kfold-00/"
phase_XRD_class_RF=f"{run_dir}campaign09c/Class_XRD/Class_XRD-0001/iter-000/kfold-00/"
phase_EBSD_class_SVM=f"{run_dir}campaign09c/Class_EBSD/Class_EBSD-0000/iter-000/kfold-00/"
phase_XRD_class_SVM=f"{run_dir}campaign09c/Class_XRD/Class_XRD-0000/iter-000/kfold-00/"

fit_E_NN=f"{run_dir}campaign09r/Fit_E/Fit_E-0007/iter-000/kfold-00/"
fit_E_RF=f"{run_dir}campaign09r/Fit_E/Fit_E-0006/iter-000/kfold-00/"
fit_E_SVM=f"{run_dir}campaign09r/Fit_E/Fit_E-0008/iter-000/kfold-00/"
fit_H_NN=f"{run_dir}campaign09r/Fit_H/Fit_H-0007/iter-000/kfold-00/"
fit_H_RF=f"{run_dir}campaign09r/Fit_H/Fit_H-0006/iter-000/kfold-00/"
fit_H_SVM=f"{run_dir}campaign09r/Fit_H/Fit_H-0008/iter-000/kfold-00/"
fit_CI_NN=f"{run_dir}campaign09r/Fit_CI/Fit_CI-0001/iter-000/kfold-00/"
fit_CI_RF=f"{run_dir}campaign09r/Fit_CI/Fit_CI-0000/iter-000/kfold-00/"
fit_CI_SVM=f"{run_dir}campaign09r/Fit_CI/Fit_CI-0002/iter-000/kfold-00/"
fit_IQ_NN=f"{run_dir}campaign09r/Fit_IQ/Fit_IQ-0001/iter-000/kfold-00/"
fit_IQ_RF=f"{run_dir}campaign09r/Fit_IQ/Fit_IQ-0000/iter-000/kfold-00/"
fit_IQ_SVM=f"{run_dir}campaign09r/Fit_IQ/Fit_IQ-0002/iter-000/kfold-00/"

# Group per kind of model
models_NN=[phase_EBSD_class_NN,phase_XRD_class_NN,fit_CI_NN,fit_IQ_NN,fit_E_NN,fit_H_NN]
models_RF=[phase_EBSD_class_RF,phase_XRD_class_RF,fit_CI_RF,fit_IQ_RF, fit_E_RF, fit_H_RF]
models_SVM=[phase_EBSD_class_SVM,phase_XRD_class_SVM,fit_CI_SVM,fit_IQ_SVM, fit_E_SVM, fit_H_SVM]

## Prepare dataframes that will gather compositions and predictions 
Dataframe is named `compo_prediction` and the columns names must correcpond to the different predictions the models are going to give.

In [14]:
compo_prediction=compo2predict.copy()
predictions_type=['Phase prediction from EBSD class', 'Phase prediction from XRD class', 'CI prediction', 'IQ prediction', 'Young Modulus prediction (GPa)', 'Hardness prediction (GPa)' ]

## Predictions

### Neural Network

In [19]:
i=0
for model_path in models_NN:
     loaded_model=keras.models.load_model(model_path+'bestmodel.h5',compile = False)
     with open(model_path+'about.json') as fd:
         about_s = json.load(fd)
     mean=about_s['monitoring']['mean']
     std=about_s['monitoring']['std']
     loaded_model.summary()
     print("Loaded.")

     compo2predict_norm=(compo2predict-mean)/std
     prediction=loaded_model.predict(compo2predict_norm)
     if "Class" in model_path: 
          prediction=[ 0 if y<0.5 else 1 for y in prediction] 

     column_name='NN - '+predictions_type[i]
     compo_prediction[column_name]=prediction
     i+=1
     del loaded_model
display(compo_prediction)
compo_prediction.to_csv('./prediction_NN.csv')

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 50)                300       
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 18,151
Trainable params: 18,151
Non-trainable params: 0
_________________________________________________________________
Loaded.
Model: "sequential_2"
_________________________________________________________________
Layer (type)          

,Zr,Nb,Mo,Ti,Cr,RF - Phase prediction from EBSD class,RF - Phase prediction from XRD class,RF - CI prediction,RF - IQ prediction,RF - Young Modulus prediction (GPa),...,SVM - Phase prediction from EBSD class,SVM - Phase prediction from XRD class,SVM - CI prediction,SVM - IQ prediction,NN - Phase prediction from EBSD class,NN - Phase prediction from XRD class,NN - CI prediction,NN - IQ prediction,NN - Young Modulus prediction (GPa),NN - Hardness prediction (GPa)
0,0.0,0.0,0.0,0.0,100.0,1.0,1,0.227018,57893.368438,267.790417,...,1.0,1,0.384302,51649.948504,1,1,0.095244,49592.136719,278.939880,11.034189
1,0.0,0.0,0.0,2.0,98.0,1.0,1,0.226734,57893.368438,236.930690,...,1.0,1,0.320717,51297.540607,1,1,0.057085,48823.363281,272.603363,11.720722
2,0.0,0.0,0.0,4.0,96.0,1.0,1,0.226929,59560.721745,235.365696,...,1.0,1,0.269445,50961.779274,1,1,0.031224,48134.925781,266.309814,12.411344
3,0.0,0.0,0.0,6.0,94.0,1.0,1,0.228720,58880.630167,234.382639,...,1.0,1,0.229274,50642.452232,1,1,0.026083,47496.328125,260.623779,12.617285
4,0.0,0.0,0.0,8.0,92.0,1.0,1,0.232064,59234.431667,241.943527,...,1.0,1,0.198448,50339.347208,1,1,0.027115,47041.703125,255.885315,12.590228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316246,98.0,0.0,0.0,0.0,2.0,0.0,1,0.019261,36816.220090,126.529137,...,0.0,1,0.032514,35646.294863,0,1,0.001217,44916.746094,157.892807,10.690042
316247,98.0,0.0,0.0,2.0,0.0,0.0,1,0.019261,36816.220090,126.529137,...,0.0,1,0.027036,35660.517383,0,1,0.002242,45371.324219,161.274643,10.751777
316248,98.0,0.0,2.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,...,0.0,1,0.037670,36809.150771,0,1,-0.000232,45376.925781,159.094711,10.655119
316249,98.0,2.0,0.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,...,0.0,1,0.026654,36710.642241,0,1,0.001476,45608.792969,161.384323,10.796916


### Random Forest

In [17]:
i=0
for model_path in models_RF:
     
     loaded_model=joblib.load(model_path+'savedmodel.joblib')
     with open(model_path+'about.json') as fd:
         about_s = json.load(fd)
     mean=about_s['monitoring']['mean']
     std=about_s['monitoring']['std']
     print("Loaded.")

     compo2predict_norm=(compo2predict-mean)/std
     prediction=loaded_model.predict(compo2predict_norm)
    
     column_name='RF - '+predictions_type[i]
     compo_prediction[column_name]=prediction
     i+=1
     del loaded_model
display(compo_prediction)
compo_prediction.to_csv('./prediction_RF.csv')

/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loaded.


/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loaded.


/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loaded.


/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator DecisionTreeRegressor from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator RandomForestRegressor from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loaded.
Loaded.
Loaded.


,Zr,Nb,Mo,Ti,Cr,RF - Phase prediction from EBSD class,RF - Phase prediction from XRD class,RF - CI prediction,RF - IQ prediction,RF - Young Modulus prediction (GPa),RF - Hardness prediction (GPa)
0,0.0,0.0,0.0,0.0,100.0,1.0,1,0.227018,57893.368438,267.790417,11.316550
1,0.0,0.0,0.0,2.0,98.0,1.0,1,0.226734,57893.368438,236.930690,11.509728
2,0.0,0.0,0.0,4.0,96.0,1.0,1,0.226929,59560.721745,235.365696,11.496725
3,0.0,0.0,0.0,6.0,94.0,1.0,1,0.228720,58880.630167,234.382639,11.411974
4,0.0,0.0,0.0,8.0,92.0,1.0,1,0.232064,59234.431667,241.943527,11.410432
...,...,...,...,...,...,...,...,...,...,...,...
316246,98.0,0.0,0.0,0.0,2.0,0.0,1,0.019261,36816.220090,126.529137,7.808660
316247,98.0,0.0,0.0,2.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660
316248,98.0,0.0,2.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660
316249,98.0,2.0,0.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,7.808660


### SVM

In [22]:
i=0
for model_path in models_SVM:
     loaded_model=joblib.load(model_path+'savedmodel.joblib')
     with open(model_path+'about.json') as fd:
         about_s = json.load(fd)
     mean=about_s['monitoring']['mean']
     std=about_s['monitoring']['std']
     print("Loaded.")

     compo2predict_norm=(compo2predict-mean)/std
     prediction=loaded_model.predict(compo2predict_norm)
    
     column_name='SVM - '+predictions_type[i]
     compo_prediction[column_name]=prediction
     i+=1
     del loaded_model
display(compo_prediction)
compo_prediction.to_csv('./prediction_SVM.csv')

/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loaded.


/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loaded.


/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator NuSVR from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loaded.


/Users/elisegarel/opt/anaconda3/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator NuSVR from version 0.24.2 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Loaded.
Loaded.
Loaded.


,Zr,Nb,Mo,Ti,Cr,RF - Phase prediction from EBSD class,RF - Phase prediction from XRD class,RF - CI prediction,RF - IQ prediction,RF - Young Modulus prediction (GPa),...,SVM - CI prediction,SVM - IQ prediction,NN - Phase prediction from EBSD class,NN - Phase prediction from XRD class,NN - CI prediction,NN - IQ prediction,NN - Young Modulus prediction (GPa),NN - Hardness prediction (GPa),SVM - Young Modulus prediction (GPa),SVM - Hardness prediction (GPa)
0,0.0,0.0,0.0,0.0,100.0,1.0,1,0.227018,57893.368438,267.790417,...,0.384302,51649.948504,1,1,0.095244,49592.136719,278.939880,11.034189,253.928163,4.396952
1,0.0,0.0,0.0,2.0,98.0,1.0,1,0.226734,57893.368438,236.930690,...,0.320717,51297.540607,1,1,0.057085,48823.363281,272.603363,11.720722,253.849879,6.745225
2,0.0,0.0,0.0,4.0,96.0,1.0,1,0.226929,59560.721745,235.365696,...,0.269445,50961.779274,1,1,0.031224,48134.925781,266.309814,12.411344,252.778293,8.739378
3,0.0,0.0,0.0,6.0,94.0,1.0,1,0.228720,58880.630167,234.382639,...,0.229274,50642.452232,1,1,0.026083,47496.328125,260.623779,12.617285,250.669495,10.353592
4,0.0,0.0,0.0,8.0,92.0,1.0,1,0.232064,59234.431667,241.943527,...,0.198448,50339.347208,1,1,0.027115,47041.703125,255.885315,12.590228,247.517345,11.579754
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
316246,98.0,0.0,0.0,0.0,2.0,0.0,1,0.019261,36816.220090,126.529137,...,0.032514,35646.294863,0,1,0.001217,44916.746094,157.892807,10.690042,167.773504,3.331968
316247,98.0,0.0,0.0,2.0,0.0,0.0,1,0.019261,36816.220090,126.529137,...,0.027036,35660.517383,0,1,0.002242,45371.324219,161.274643,10.751777,171.867932,4.336146
316248,98.0,0.0,2.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,...,0.037670,36809.150771,0,1,-0.000232,45376.925781,159.094711,10.655119,167.786943,3.540028
316249,98.0,2.0,0.0,0.0,0.0,0.0,1,0.019261,36816.220090,126.529137,...,0.026654,36710.642241,0,1,0.001476,45608.792969,161.384323,10.796916,170.759134,4.661567
